# Deducing Struct Layout with Symbolic Execution
The goal of this tutorial is to determine the input format
accepted by the test program `dns.{arch}.elf`
without reading the source or resorting to manual RE.

We'll use memory labelling to lay out a hypothetical message structure
in our machine state, and then use `FieldDetectionAnalysis` to
detect inconsistencies between our hypothesis and how the code accesses the data.
By iterating this process, we will refine our hypothesis until it matches
what the code accepts.

## Setup & Installation
Install the required packages as described in the install directions.

You can build the example programs by running `make all` from this directory.
This requires the cross-compiler toolchains installed as part of the apt dependencies;
this analysis will work on any architecture supported by angr.

Let's test this program against the example input `examples/answer.dat`:

```
$> hexdump -C examples/answer.dat
00000000  ce 70 81 80 00 01 00 01  00 00 00 01 06 67 6f 6f  |.p...........goo|
00000010  67 6c 65 03 63 6f 6d 00  00 01 00 01 c0 0c 00 01  |gle.com.........|
00000020  00 01 00 00 00 d2 00 04  8e fa 41 ce 00 00 29 04  |..........A...).|
00000030  c4 00 00 00 00 00 1c 00  0a 00 18 cb 6e 4b 21 27  |............nK!'|
00000040  ca a2 eb 8f fc 9f 64 67  85 5f 7f cf 3e 38 e4 56  |......dg._..>8.V|
00000050  80 72 f9                                          |.r.|
00000053

$> ./bin/dns.amd64.elf examples/answer.dat
DNS Message:
        TID:          ce70
        FLAGS:        0120
        # Questions:  1
        # Answers:    0
        # Auth. Rs:   0
        # Extra Rs:   1
Question:
        Name:         google.com
        Type:         1
        Class:        1
Record:
        Name:
        Type:         41
        Class:        1232
```

If it's not apparent yet, this program parses basic DNS messages.
However, the exact structure accepted by the parser isn't obvious.
We could decompile the binary and try to elicit the structure but 
we are going to use SmallWord analysis capabilites to figure it out.

## Harnessing the Parser

For sanity's sake, let's assume we know that the parser is implemented in `parse_dns_msg()`,
and that we roughly know its arguments from `main()`:

- **Arg 1:** `uint8_t *` pointing to an input buffer
- **Arg 2:** `size_t` holding its size
- **Arg 3:** `size_t *` pointing to a value starting at zero
- **Arg 4:** `struct *` pointing to an unknown struct

Our goal for this exercise is to figure out the structure of argument 1,
which we'll call `buf`, and argument 4, which we'll call `msg`.

First, we set up some smallworld boilerplate.
This configures logging, as well as the hinter
that will let us see the output from our analysis. 

In [1]:
import warnings

warnings.filterwarnings(
    "ignore", message="Protobuf gencode version .* is exactly one major version older.*"
)

In [2]:
import logging
import pathlib
import claripy  # from angr, an abstracted constraint-solving wrapper
import timeout_decorator  # to allow capping execution time for symbolic analysis
import smallworld
import smallworld.analyses.field_detection
import smallworld.analyses.unstable.angr.visitor
from smallworld.analyses.field_detection import FieldDetectionAnalysis

# Set up logging and hinting
smallworld.logging.setup_logging(level=logging.INFO, force_colors=True)
smallworld.hinting.setup_hinting(stream=True, verbose=True, force_colors=True)

log = logging.getLogger("smallworld")

Next, we create the basic machine state, load our ELF file into a smallworld state object, and add the code to the machine.

Since we don't expect a specific platform, we can use the platform definition parsed out of the ELF headers.

Also, since this is the only ELF we're loading, we can use its notion of program bounds to define which memory the analysis can execute.

In [3]:
# Create a machine
machine = smallworld.state.Machine()

# Load the code
filepath = "bin/dns.amd64.elf"
with open(filepath, "rb") as f:
    code = smallworld.state.memory.code.Executable.from_elf(f, 0x40000)
    machine.add(code)

# Apply the code's bounds to the machine
for bound in code.bounds:
    machine.add_bound(bound[0], bound[1])

# Use the ELF's notion of the platform
platform = code.platform

# Create the analysis; we'll need it later.
analysis = FieldDetectionAnalysis(platform)

[+] Address: 40000
[+] {"time": "2025-09-05 14:15:14,064", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:14,065", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:14,066", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:14,066", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}


In [4]:
# Create a CPU
cpu = smallworld.state.cpus.CPU.for_platform(platform)
machine.add(cpu)

# Use lief to find the address of parse_dns_message
sym = code.get_symbol_value("parse_dns_message")
cpu.rip.set(sym)

# Add a blank stack
stack = smallworld.state.memory.stack.Stack.for_platform(
    platform, 0x7FFFFFFFC000, 0x4000
)
machine.add(stack)

stack.push_integer(0x01010101, 8, None)

# Configure the stack pointer
sp = stack.get_pointer()
cpu.rsp.set(sp)

## Setting up memory
Now that we have the program set up, we can start to model the inputs to `parse_dns_message()`.
We know we need a message struct, a memory buffer, and a `size_t`
that are all passed by reference.

For ease of modelling, let's just allocate them all as globals.

To start, we'll allocate `buf` and `msg` as blank swaths of memory.
As we learn more about their internal structure from the analysis,
we will come back and refine this part of our harness.

In [5]:
# Configure somewhere for arguments to live
gdata = smallworld.state.memory.Memory(0x6000, 0x1000)
machine.add(gdata)
# DNS message struct
# Sort of cheating that I know how big it is.
gdata[0] = smallworld.state.SymbolicValue(48, None, None, "msg")
# Input buffer
gdata[48] = smallworld.state.SymbolicValue(512, None, None, "buf")
# Offset into buffer
gdata[560] = smallworld.state.IntegerValue(0, 8, "off", False)

Next, we use what we know about the function signature to assign values to registers.
We know that the first arg points to `buf`, the second is the size of `buf`,
the third points to `off`, and the fourth points to `msg`.

In [6]:
# Configure arguments
# arg 0: pointer to buf
cpu.rdi.set(gdata.address + 48)
cpu.rdi.set_label("PTR buf")
# arg 1: buffer capacity
cpu.rsi.set(512)
cpu.rsi.set_label("cap")
# arg 2: pointer to off
cpu.rdx.set(gdata.address + 560)
cpu.rdx.set_label("PTR off")
# arg 3: pointer to msg
cpu.rcx.set(gdata.address)
cpu.rcx.set_label("PTR msg")

Because we'll be using symbolic analysis, we'll want the option to set a timeout on analysis, in case the state explodes. We'll define an `analysis_with_timeout` function here

In [7]:
# We're going to create the option to pass a timeout when doing analysis
@timeout_decorator.timeout(120)  # argument is in seconds
def analyze_with_timeout(machine, analysis):
    try:
        machine.analyze(analysis)
    except timeout_decorator.timeout_decorator.TimeoutError:
        print("Analysis timeout")

At long last, we're reading to actually run our analysis, this time we won't have to worry about state explosion.

In [8]:
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6030: <smallworld.state.state.SymbolicValue object at 0x7568b6c4e4a0> := msg
[+] {"time": "2025-09-05 14:15:14,110", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 48, "label": "msg", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x7568b6c4d060> := buf
[+] {"time": "2025-09-05 14:15:14,112", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:14,114", "level": "INFO", "source": "smal

_The work so far is captured in `dns_0.py`_

## Deducing Accessed Fields
The analysis produced by `FieldDetectionAnalysis` yields the following hint:

```
[*] Partial write to known field
[*]   address:    0x6010
[*]   size:       8
[*]   pc:         0x41e73
[*]   access:     write
[*]   guards:
[*]   field:      msg[16:24]
[*]   expr:       <BV64 0x0>
```
The code at `0x41e73` tried to write 8 bytes of zero starting 16 bytes into our `msg` buffer.
But we defined `msg` as a 48 byte contiguous region and didn't expect to see a field.  

With this information in hand we can redefine `msg` as the following


In [9]:
gdata[0] = smallworld.state.SymbolicValue(
    16, None, None, "msg.hdr.unk"
)  # we're going to assume the first bytes are a header
gdata[16] = smallworld.state.SymbolicValue(8, None, None, "msg.a")
gdata[24] = smallworld.state.SymbolicValue(24, None, None, "msg.unk")

We're not sure what `msg.a` is, but we know it's very likely a field.
We're on our way!

Running our updated script gives a very similar hint to the first:

In [10]:
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6010: <smallworld.state.state.SymbolicValue object at 0x756894bb2a40> := msg.hdr.unk
[+] {"time": "2025-09-05 14:15:14,535", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 16, "label": "msg.hdr.unk", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x7568b6c4d060> := buf
[+] {"time": "2025-09-05 14:15:14,536", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:14,537", "level": "INFO",

Running our updated script gives a very similar hint to the first:

```
[*] Partial write to known field
[*]   address:    0x6018
[*]   size:       8
[*]   pc:         0x41e7f
[*]   access:     write
[*]   guards:
[*]   field:      msg.unk[0:8]
[*]   expr:       <BV64 0x0>
```

In fact, four iterations tell us the same thing;
the program writes zero to four 8-byte fields right next to each other:


In [11]:
gdata[0] = smallworld.state.SymbolicValue(16, None, None, "msg.hdr.unk")
gdata[16] = smallworld.state.SymbolicValue(8, None, None, "msg.a")
gdata[24] = smallworld.state.SymbolicValue(8, None, None, "msg.b")
gdata[32] = smallworld.state.SymbolicValue(8, None, None, "msg.c")
gdata[40] = smallworld.state.SymbolicValue(8, None, None, "msg.d")

In [12]:
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6010: <smallworld.state.state.SymbolicValue object at 0x75689323beb0> := msg.hdr.unk
[+] {"time": "2025-09-05 14:15:14,943", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 16, "label": "msg.hdr.unk", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6230: <smallworld.state.state.SymbolicValue object at 0x7568b6c4d060> := buf
[+] {"time": "2025-09-05 14:15:14,944", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 512, "label": "buf", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:14,945", "level": "INFO",

Continuing this process we first find that we have a 2 byte regions of `buf` referenced. Let's label it and move forward.

Also remember, we'll also need to redefine the memory _after_ this 2 byte variable so that smallworld is tracking the memory space and will alert us to accesses.

In [13]:
gdata[48] = smallworld.state.SymbolicValue(2, None, None, "buf.a")
gdata[50] = smallworld.state.SymbolicValue(510, None, None, "buf")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6010: <smallworld.state.state.SymbolicValue object at 0x75689323beb0> := msg.hdr.unk
[+] {"time": "2025-09-05 14:15:15,759", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 16, "label": "msg.hdr.unk", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x756894b26920> := buf.a
[+] {"time": "2025-09-05 14:15:15,761", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:15,762", "level": "INFO

Iterating on this process we find that we have multiple 2 byte regions which we identify as parts of `msg.hdr`, since we suspect it's the part of the struct representing the DNS header.  The information in the final 4 bytes of this region are never referenced.  We leave them as a final 4 byte region we call `msg.hdr.unused`:

In [14]:
gdata[0] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.a")
gdata[2] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.b")
gdata[4] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.c")
gdata[6] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.d")
gdata[8] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.e")
gdata[10] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.f")
gdata[12] = smallworld.state.SymbolicValue(4, None, None, "msg.hdr.unk")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x75689320e3e0> := msg.hdr.a
[+] {"time": "2025-09-05 14:15:16,662", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x756894b26920> := buf.a
[+] {"time": "2025-09-05 14:15:16,664", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:16,665", "level": "INFO", "s

Repeating this process for the remaining part of `buf` we can further identify regions which we summarize as:

In [15]:
gdata[50] = smallworld.state.SymbolicValue(2, None, None, "buf.b")
gdata[52] = smallworld.state.SymbolicValue(2, None, None, "buf.c")
gdata[54] = smallworld.state.SymbolicValue(2, None, None, "buf.d")
gdata[56] = smallworld.state.SymbolicValue(2, None, None, "buf.e")
gdata[58] = smallworld.state.SymbolicValue(2, None, None, "buf.f")
gdata[60] = smallworld.state.SymbolicValue(500, None, None, "buf")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x40000 - 0x46000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x75689320e3e0> := msg.hdr.a
[+] {"time": "2025-09-05 14:15:18,035", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x756894b26920> := buf.a
[+] {"time": "2025-09-05 14:15:18,038", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:18,039", "level": "INFO", "s

_The work so far is captured in `dns_1.py`_

## Bringing in Malloc
Notice that the `FieldDetectionAnalysis` routine no longer provides information about a partial access to a known field, so execution continues.  The last successful read is reported to be at instruction `0x41ed2`, and that the analysis ends there with the last known good state being reported as:

```
[!] State at <BV64 0x1076>:
```

We loaded our ELF at `0x40000` so something's way off.
If we disassemble the code and look around `0x1ed2` (which is `0x41ed2` minus the loading address `0x40000`) we find:

```
$> objdump -d -R ./bin/dns.amd64.elf
...
    1ed2:       0f b7 40 04             movzwl 0x4(%rax),%eax
    1ed6:       48 69 f8 08 01 00 00    imul   $0x108,%rax,%rdi
    1edd:       e8 8e f1 ff ff          call   1070 <malloc@plt>
...
```

This is an external call to `malloc()` which we have not loaded.

We have not yet discussed how to handle an external call like this.  One strategey would be to stub out `malloc()` and carry on.  Howerver, we would like to know what is happening inside the heap-allocated buffers as well. Luckily, `FieldDetectionAnalysis` comes with a malloc model.

Let's update our harness:

_Note: At this point we recreate our machine so any changes we made along the way are reset._

In [16]:
def reinitialize_demo():
    # reinitializing outside of this routine
    global machine
    global platform
    global analysis
    global cpu
    global sym
    global stack
    global sp
    global code

    # Create a machine
    machine = smallworld.state.Machine()

    # Load the code
    filepath = "bin/dns.amd64.elf"
    with open(filepath, "rb") as f:
        code = smallworld.state.memory.code.Executable.from_elf(f, 0x40000)
        machine.add(code)

    # Apply the code's bounds to the machine
    for bound in code.bounds:
        machine.add_bound(bound[0], bound[1])

    # Use the ELF's notion of the platform
    platform = code.platform

    # Create the analysis; we'll need it later.
    analysis = FieldDetectionAnalysis(platform)

    # Create a CPU
    cpu = smallworld.state.cpus.CPU.for_platform(platform)
    machine.add(cpu)

    # Use lief to find the address of parse_dns_message
    sym = code.get_symbol_value("parse_dns_message")
    cpu.rip.set(sym)

    # Add a blank stack
    stack = smallworld.state.memory.stack.Stack.for_platform(
        platform, 0x7FFFFFFFC000, 0x4000
    )
    machine.add(stack)
    machine.add(gdata)

    stack.push_integer(0x01010101, 8, None)

    # Configure the stack pointer
    sp = stack.get_pointer()
    cpu.rsp.set(sp)

    # Configure arguments
    # arg 0: pointer to buf
    cpu.rdi.set(gdata.address + 48)
    cpu.rdi.set_label("PTR buf")
    # arg 1: buffer capacity
    cpu.rsi.set(512)
    cpu.rsi.set_label("cap")
    # arg 2: pointer to off
    cpu.rdx.set(gdata.address + 560)
    cpu.rdx.set_label("PTR off")
    # arg 3: pointer to msg
    cpu.rcx.set(gdata.address)
    cpu.rcx.set_label("PTR msg")


reinitialize_demo()

[+] Address: 40000
[+] {"time": "2025-09-05 14:15:21,237", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:21,239", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:21,240", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:21,240", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}


In [17]:
from smallworld.analyses.field_detection import MallocModel, FreeModel

# Add a blank heap
heap = smallworld.state.memory.heap.BumpAllocator(0x20000, 0x10000)
machine.add(heap)

# Configure malloc and free models
malloc = MallocModel(
    0x10000, heap, platform, analysis.mem_read_hook, analysis.mem_write_hook
)
machine.add(malloc)
machine.add_bound(malloc._address, malloc._address + 16)

free = FreeModel(0x1036)
machine.add(free)
machine.add_bound(free._address, free._address + 16)

This will invoke a function hook mimicking `malloc()` if we call address `x10000`. If we were in a raw binary, we'd have to do some manual patching to tell it where we put `malloc()`, but this is an ELF.

Let's use the relocation entries to our advantage:

In [18]:
# Apply relocations to malloc
code.update_symbol_value("malloc", malloc._address)

In [19]:
machine.analyze(analysis)

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x75689320e3e0> := msg.hdr.a
[+] {"time": "2025-09-05 14:15:21,267", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x756894b26920> := buf.a
[+] {"time": "2025-09-05 14:15:21,268", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:21,270", "level": "INFO", "s

_The work so far is captured in `dns_2.py`_

This ran for much longer, but eventually halted with an error similar to this:

```
[*] Capacity did not collapse to one value
[*] The following fields are lengths:
[*]    msg.hdr.c_119_16
[*] Concretize them to continue analysis
```

This output indicates that the analysis was able to deduce that the `msg.hdr.c` field gives the numbe of items allocated in our buffer.  In fact, if we scroll up to instruction `0x418d0` we see that `msg.hdr.c` gets filled in by the contents of `buf.c`. 

In order to concretize `msg.hdr.c` we need to assign a concrete value to `buf.c` (and give it a new name):

In [20]:
gdata[52] = smallworld.state.BytesValue(b"\x00\x01", "buf.msg.a.len")
machine.analyze(FieldDetectionAnalysis(platform))

[!] 0x7fffffffc000 - 0x800000000000
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x75689320e3e0> := msg.hdr.a
[+] {"time": "2025-09-05 14:15:24,442", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24576, "size": 2, "label": "msg.hdr.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6030 - 0x6032: <smallworld.state.state.SymbolicValue object at 0x756894b26920> := buf.a
[+] {"time": "2025-09-05 14:15:24,443", "level": "INFO", "source": "smallworld.analyses.field_detection.field_analysis", "content": {"message": "Tracking new field", "address": 24624, "size": 2, "label": "buf.a", "class": "smallworld.analyses.field_detection.hints.TrackedFieldHint"}}
[!]   0x6230 - 0x6238: <smallworld.state.state.IntegerValue object at 0x7568b6c4c280> := off
[+] {"time": "2025-09-05 14:15:24,444", "level": "INFO", "s

## Analyzing the behavior of malloc

Scrolling up to the malloc call after the memory read at `0x41ed2` we see the following output from the malloc:

```
[!] malloc: Alloc'd 264 bytes at 0x20010
[+] Storing 8 bytes at 0x20008
[!]   Length field msg.hdr.c not known
[+] 0x41ee6: WRITING 0x6010 - 0x6018 (msg.a)
[+]   <BV64 0x20010>
```

Let's take this one line at a time:

- `malloc: Alloc'd 264 bytes at 0x20010`:  Since we assigned `0x0001` to the length in `buf.c`, this line is telling us that space for the struct is 264 bytes (the `malloc()` model would  warn us if the length computation was more complex than "variable * magnitude".)

- `Storing 8 bytes at 0x20008`: Malloc returns a pointer to the newly allocated memory so the address of the heap memory is stored there.

- `Length field msg.hdr.c not known`: We haven't told the `malloc()` model that it should track structs allocated using `msg.hdr.c`, or what they should look like.

- `0x41ee6: WRITING 0x6010 - 0x6018 (msg.a)`: The pointer to our new heap array is stored in `msg.a`, making `msg.hdr.c` the length of `msg.a`. Let's update our harness with a better name for this pointer.

Let's go ahead and define a more clear variable name, since it appears the `len` value is being used to determine how many chunks of memory to allocate. This seems like behaviour we'd expect when allocating structs, so let's update to reflect that.

We wrap up this section by noting that execution stopped on another read of a single byte from a larger buffer.  We'll finish this treatment of models with four final actions:
- combine all the model work into a routine that can be called when we reinitialize our analysis
- update our harness to take account of the single byte, we'll assume this is part of a struct
- update our variable names to better reflect usage
- run again

In [21]:
def add_malloc_free_models():
    global code
    global malloc
    global free
    global heap

    # Add a blank heap
    heap = smallworld.state.memory.heap.BumpAllocator(0x20000, 0x10000)
    machine.add(heap)

    # Configure malloc and free models
    malloc = MallocModel(
        0x10000, heap, platform, analysis.mem_read_hook, analysis.mem_write_hook
    )
    machine.add(malloc)
    machine.add_bound(malloc._address, malloc._address + 16)

    free = FreeModel(0x1036)
    machine.add(free)
    machine.add_bound(free._address, free._address + 16)

    # Apply relocations to malloc
    code.update_symbol_value("malloc", malloc._address)


reinitialize_demo()
add_malloc_free_models()
gdata[4] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.numstructs")
gdata[52] = smallworld.state.BytesValue(b"\x00\x01", "buf.msg.hdr.numstructs")
gdata[60] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.var.a")
gdata[61] = smallworld.state.SymbolicValue(499, None, None, "buf")
machine.analyze(analysis)

[+] Address: 40000
[+] {"time": "2025-09-05 14:15:28,638", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:28,639", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:28,639", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:28,640", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7

# DNS exploration: Next control field

The end of our last run produced new output.  In particular, it noted that `buf.struct.var.a` seems to be
influencing whether the next two bytes of `buf` are a single field or multiple fields.  What conditions are making this happen?  To address this question, we look at the various reports of `Partial access to known field` and look at the conditions that were required for that partial access to occurr.  This happens in three cases.  Extracting just the information relevent to `buf.struct.var.a` we have:

### Case 1:
```
[*] Partial access to known field
[*]   address:    0x603d
[*]   size:       1
[*]   pc:         0x415c8
[*]   access:     read
[*]   guards:
[*]     0x41592: <Bool buf.struct.var.a != 0> [[<BV8 buf.struct.var.a>]]
[*]     0x415ab: <Bool (buf.struct.var.a & 192) == 192> [[<BV8 buf.struct.var.a>]]
[*]   field:      buf[0:1]
```
We get to this case exactly when the two high bits set on `buf.struct.var.a`.  So since its a byte, `buf.struct.var.a` is 64 and 255 inclusive.

### Case 2:
```
[*] Partial access to known field
[*]   address:    0x603d
[*]   size:       2
[*]   pc:         0x41a44
[*]   access:     read
[*]   guards:
[*]     0x41592: <Bool buf.struct.var.a == 0> [[<BV8 buf.struct.var.a>]]
[*]   field:      buf[0:2]
```
We get to this case exactly when `buf.struct.var.a == 0`.

### Case 3:
```
[*] Partial access to known field
[*]   address:    0x603d
[*]   size:       1
[*]   pc:         0x416b9
[*]   access:     read
[*]   guards:
[*]     0x41592: <Bool buf.struct.var.a != 0> [[<BV8 buf.struct.var.a>]]
[*]     0x415ab: <Bool (buf.struct.var.a & 192) != 192> [[<BV8 buf.struct.var.a>]]
[*]     0x4161c: <Bool (buf.struct.var.a & 192) == 0> [[<BV8 buf.struct.var.a>]]
[*]     0x41638: <Bool ((if (0#32 .. (0#24 .. buf.struct.var.a)) < 0xff then 1 else 0) | (if buf.struct.var.a == 255 then 1 else 0)) != 0> [[<BV8 buf.struct.var.a>]]
[*]     0x4165c: <Bool cap - off_184_64 >= (0#32 .. (0#24 .. buf.struct.var.a))> [[<BV8 buf.struct.var.a>, <BV64 off_184_64>, <BV64 cap>]]
[*]     0x416a4: <Bool 0x0 < (0#32 .. (0#24 .. buf.struct.var.a))> [[<BV8 buf.struct.var.a>]]
[*]   field:      buf[0:1]
```
The conditions on `buf.struct.var.a` look much more complicated here but with a little work it is not hard to see that this case arises exactly when the two high bits are unset on `buf.struct.var.a` and `buf.struct.var.a` is non-zero.  Since its a byte, `buf.struct.var.a` is between 1 and 63 inclusive.

Above we can see that in Case 1 and 3, the routine tries to read `buf[0:1]` as a field, and in Case 2, it tries to read `buf[0:2]` as a field. Expirimenting with Case 1 will show us that it exits the routine, only Case 2 and 3 continue parsing.

For exploration purposes, let's expiriment with Case 3, where `buf[0:1]` is interperted as a field. Since only one byte is being interperted as a field, this may be the simpler case to begin with. If this analysis were to finish and be unsatisfactory, it is easily possible to come back and continue digging into the other cases and how they cause the buffer to be interperted. However, since we can only explore one possibility at a time, we're going to choose Case 3.

First, we'll need to add a constraint that `buf.struct.var.a` will be greater than 0. Recall UGT here signifies an unsigned greater than operation, and we specify that buf.struct.var.a must be greater than the integer on the right, in this case we make this 0. It will then incriment to 1, since 1 is greater than 0. If you wanted to test Case 1, you could instead set the right side of this UGT argument to `claripy.BVV(64, 8)`.

```python
claripy.UGT(claripy.BVS("buf.struct.var.a", 8, explicit_name=True), claripy.BVV(0, 8))
```

We're also going to need to add the ability to track malloc'd arrays of items. We know from our last run that `msg.hdr.numstructs` is responsible for controlling how many homogenous structs of size 264 are malloc'd. We'll tell the program that we're going to allocate a number of structs based on `msg.hdr.numstructs`, that the struct will be of size 264, and if you see it to label it unknown (`unk`). Recall previously we defined `msg.hdr.numstructs` statically to equal `0x0001` so we will always get a single allocation of the struct.

```python
malloc.bind_length_to_struct("msg.hdr.numstructs", "msg.struct.item", [(264, "unk")])
```

Also, because we are now reading the first field of `buf` as it's own field, we'll define it seperatly. We also know the second byte can be defined seperatly, so we'll define it as well. We'll leave the rest of the buffer in tact for now.

```python
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.var.b")
gdata[62] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.var.c")
gdata[63] = smallworld.state.SymbolicValue(497, None, None, "buf")
```

In [22]:
reinitialize_demo()
add_malloc_free_models()

gdata[4] = smallworld.state.SymbolicValue(2, None, None, "msg.hdr.numstructs")
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.var.b")
gdata[62] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.var.c")
gdata[63] = smallworld.state.SymbolicValue(497, None, None, "buf")

malloc.bind_length_to_struct("msg.hdr.numstructs", "msg.struct.item", [(264, "unk")])

machine.add_constraint(
    claripy.UGT(
        claripy.BVS("buf.struct.var.a", 8, explicit_name=True), claripy.BVV(0, 8)
    )
)


# We're going to create the option to pass a timeout when doing analysis
@timeout_decorator.timeout(180)  # argument is in seconds
def analyze_with_timeout(machine, analysis):
    try:
        machine.analyze(analysis)
    except timeout_decorator.timeout_decorator.TimeoutError:
        print("Analysis timeout")


analyze_with_timeout(machine, analysis)

[+] Address: 40000
[+] {"time": "2025-09-05 14:15:34,367", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:34,369", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:34,369", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:34,369", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x7fffffffc000 - 0x800000000000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue

_The work so far is captured in `dns_4.py`_

You can observe many constrained writes to `0x20010`, our formerly defined struct. Then, as the program exits, we encounter a large number of deadended states. The corresponding guards and fields sections of these states provide the symbolic constraints that can help inform our understanding of what is happening in the struct. Simplfiying and understanding symbolic constraints is a topic _well_ outside of the scope of this demo, but if you spend several hours doing analysis, you will find the following:

`gdata[60]`: `buf.struct.var.a` becomes a length field, driving the length of a char buffer.

`gdata[61]`: `buf.struct.var.b` becomes the first char of the string defined above.

`gdata[62]`: `buf.struct.var.c` becomes a length field, driving the length of another buffer.

We'll rename these items below to have more informative names.

We'll remove the constraint on `buf.struct.var.a` being greater than 0 and instead set it to a static value, so we know what length to expect for the following char buffer.

It's important to note that this DNS analysis is a research proof of concept. It would be infeasible to use symbolic analysis across fields 0-255 for example, hence why we constrain to 4 chars. While the format may be the same, it would require significantly more computing power. We are only going to explore very short domain names to keep the example more concise.

```python
fields = [(1, f"text.{i}") for i in range(0, 3)]
fields.append((253, "text"))
fields.append((8, "unk"))

malloc.bind_length_to_struct("msg.hdr.numstructs", "msg.struct.item", fields)
```

We can also now label the items we've found in the struct:

```python
gdata[60] = smallworld.state.IntegerValue(1, 1, "buf.struct.string0.len")
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.string0[0]")
gdata[62] = smallworld.state.IntegerValue(0, 1, "buf.struct.string1.len")
```

In [23]:
reinitialize_demo()
add_malloc_free_models()

fields = [(1, f"text.{i}") for i in range(0, 3)]
fields.append((253, "text"))
fields.append(
    (8, "unk")
)  # We know there's 256 bytes of text, but 264 bytes of the struct total, so we block off the remaining 8 bytes

malloc.bind_length_to_struct("msg.hdr.numstructs", "msg.struct.item", fields)

gdata[60] = smallworld.state.IntegerValue(1, 1, "buf.struct.string0.len")
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.string0[0]")
gdata[62] = smallworld.state.IntegerValue(0, 1, "buf.struct.string1.len")
gdata[63] = smallworld.state.SymbolicValue(497, None, None, "buf")

analyze_with_timeout(machine, analysis)

[+] Address: 40000
[+] {"time": "2025-09-05 14:15:42,739", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:42,740", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:42,740", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:42,741", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x7fffffffc000 - 0x800000000000
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.sta

_The work so far is captured in `dns_5.py`_

Once running this, we discover there a field from a `partial access to known field` at `buf[0:2]`. With this defined, we'll discover another field, once again at `buf[0:2]`. We can define those now.

```python
gdata[64] = smallworld.state.SymbolicValue(2, None, None, "buf.struct.a")
gdata[66] = smallworld.state.SymbolicValue(2, None, None, "buf.struct.b")
gdata[68] = smallworld.state.SymbolicValue(492, None, None, "buf")
```

With just these defined, we'll find another error (partial access in `msg.a.item.unk`), that these are referenced later. Seemingly the 2-byte ints are promoted to a 4-byte int, which is why the spacing is 4 bytes for each, not 2.

These correspond to two fields following our prior struct def:

```python
fields.append((4, "a"))
fields.append((4, "b"))
```

In [24]:
reinitialize_demo()
add_malloc_free_models()

fields = [(1, f"text.{i}") for i in range(0, 3)]
fields.append((253, "text"))
fields.append((4, "a"))
fields.append((4, "b"))

malloc.bind_length_to_struct("msg.hdr.numstructs", "msg.struct.item", fields)

gdata[60] = smallworld.state.IntegerValue(2, 1, "buf.struct.string0.len")
gdata[61] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.string0[0]")
gdata[62] = smallworld.state.SymbolicValue(1, None, None, "buf.struct.string0[1]")
gdata[63] = smallworld.state.IntegerValue(0, 1, "buf.struct.string1.len")
gdata[64] = smallworld.state.SymbolicValue(2, None, None, "buf.struct.a")
gdata[66] = smallworld.state.SymbolicValue(2, None, None, "buf.struct.b")
gdata[68] = smallworld.state.SymbolicValue(492, None, None, "buf")

analyze_with_timeout(machine, analysis)

[+] Address: 40000
[+] {"time": "2025-09-05 14:15:52,621", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies interpreter b'/lib64/ld-linux-x86-64.so.2\\x00'", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:52,622", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program includes dynamic linking metadata", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:52,623", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies stack permissions", "class": "smallworld.hinting.hinting.Hint"}}
[+] {"time": "2025-09-05 14:15:52,623", "level": "INFO", "source": "smallworld.state.memory.elf.elf", "content": {"message": "Program specifies RELRO data", "class": "smallworld.hinting.hinting.Hint"}}
[!] 0x20000 - 0x30000
[!] 0x6000 - 0x7000
[!]   0x6000 - 0x6002: <smallworld.state.state.SymbolicValue object at 0x7

_All this work is captured in `dns_6.py`_

# End of Line
This is the end of the example, we've successfully parsed out embedded structs being used for DNS. If we line up answers we see running the program with the data, we can determine more of what the fields are that we've been manipulating. After you've done the above excercises, you may want to try labeling the fields yourself. Below, we include the answer key for the fields we've used so far. Some structs remain unlabeled as an excercise for the reader to define the next part of the struct.

In [25]:
fields = [(1, f"text.{i}") for i in range(0, 3)]
fields.append((253, "text"))
fields.append((4, "qtype"))
fields.append((4, "qclass"))

# DNS message struct
gdata[0] = smallworld.state.SymbolicValue(2, None, None, "mem.dns.hdr.tid")
gdata[2] = smallworld.state.SymbolicValue(2, None, None, "mem.dns.hdr.flags")
gdata[4] = smallworld.state.SymbolicValue(2, None, None, "mem.dns.hdr.n_qs")
gdata[6] = smallworld.state.SymbolicValue(2, None, None, "mem.dns.hdr.d")
gdata[8] = smallworld.state.SymbolicValue(2, None, None, "mem.dns.hdr.e")
gdata[10] = smallworld.state.SymbolicValue(2, None, None, "mem.dns.hdr.f")
# NOTE: 4 bytes of padding here; never referenced
gdata[16] = smallworld.state.SymbolicValue(8, None, None, "msg.a")
gdata[24] = smallworld.state.SymbolicValue(8, None, None, "msg.b")
gdata[32] = smallworld.state.SymbolicValue(8, None, None, "msg.c")
gdata[40] = smallworld.state.SymbolicValue(8, None, None, "msg.d")
# Input buffer
gdata[48] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.hdr.tid")
gdata[50] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.hdr.flags")
gdata[52] = smallworld.state.BytesValue(b"\x00\x01", "buf.dns.hdr.n_qs")
gdata[54] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.hdr.d")
gdata[56] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.hdr.e")
gdata[58] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.hdr.f")
# This is a sequence of zero or more run-length-encoded strings.
# Strings can be of length 0 - 63,
# and the total number of characters can't exceed 25
gdata[60] = smallworld.state.IntegerValue(2, 1, "buf.dns.q1.qname.domain0.len")
gdata[61] = smallworld.state.SymbolicValue(
    1, None, None, "buf.dns.q1.qname.domain0.char[0]"
)
gdata[62] = smallworld.state.SymbolicValue(
    1, None, None, "buf.dns.q1.qname.domain0.char[1]"
)
gdata[63] = smallworld.state.IntegerValue(0, 1, "buf.dns.q1.qname.domain1.len")
gdata[64] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.q1.qtype")
gdata[66] = smallworld.state.SymbolicValue(2, None, None, "buf.dns.q1.qclass")
gdata[68] = smallworld.state.SymbolicValue(492, None, None, "buf")
# Offset into buffer
gdata[560] = smallworld.state.IntegerValue(0, 8, "off", False)

# What's Next
The remainder of the DNS protocol remains as an excercise to the reader, there remains an entirely new struct definition. We've worked through one together, the DNS Question struct, now you can work through one yourself! If you get stuck, the new struct is fleshed out in `dns_7.py` if you want to check your answers. You can also see all the struct definitions in `dns.h`.

If you're ready to get started, there's an error above that will serve as your next clue:
```
[*] Capacity did not collapse to one value
[*] The following fields are lengths:
[*]    msg.hdr.d_334_16
[*] Concretize them to continue analysis
```